In [2]:
#!pip install --upgrade google-api-python-client

In [98]:
# API KEY
API_KEY = "ENV_"

# 1. set up comment scraper

In [89]:
from googleapiclient.discovery import build

# Build a resource object for interacting with the API
youtube = build('youtube', 'v3', developerKey=API_KEY)

def get_video_comments(video_id, max_results=1000):
    comments = []
    response = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        maxResults=max_results,
        textFormat='plainText'
    ).execute()

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        # Check if there are more comments
        if 'nextPageToken' in response:
            response = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=max_results,
                textFormat='plainText',
                pageToken=response['nextPageToken']
            ).execute()
        else:
            break

    return comments

In [ ]:
# Example Usage
video_comments = get_video_comments('1U0-nCihPDI')
#print(video_comments)

In [4]:
for comment in video_comments:
    print(comment)

Love you!
16:32 just AWESOME!!!
It was me, silly 😛🤪☎️☎️
I can't tell you how happy I am,that we are connected again! Everything was restricted 😰👁️👁️🦋
Dear James,I got a new code 😊I am happy to find you here playing your wonderful guitar 🎸 i feel happy again 💜❤️
Wonderful to be back I can sleep with a smile 🥰😀🎸🦋
Just when I am almost giving up , you rescue me again 💞🎸💟🦋😀
I will always try to find James bay, my favourite human being 🎸💓😀
I love you beautiful ❤❤🇧🇷🇧🇷
I thought he was super high, then I realised I had playback speed set to 75%.
You deserve my best 😘 smile on a busy Saturday 🙂🤣
I am always happy to see this lesson 😀😌
The first famous person I’ve ever seen that could actually teach
3:25
3:26
Im replay button bro
Beautiful!!!!
Ah! My wish had already come true. Gentleman, sir. Thank you.
3:26
I 💝 U
Sounds great but you know it’s in G starts with a Gmaj7
Thanks man! Didn’t even know you were in the neighborhood. Love your music and will definitely be in attendance.
8:45 Start ex

# 2. Sentiment analysis model

In [6]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download VADER lexicon
nltk.download('vader_lexicon')



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/imacaristorenas/nltk_data...


Original Score: 0.6696, Scaled Score: 83.48


In [99]:
# Create a Sentiment Intensity Analyzer object
sia = SentimentIntensityAnalyzer()

# Example comment
comment = "I hate this more than anything! "

# Get sentiment scores
sentiment = sia.polarity_scores(comment)

# Scale compound score from [-1, 1] to [0, 100]
scaled_score = (sentiment['compound'] + 1) * 50

print(f"Original Score: {sentiment['compound']}, Scaled Score: {scaled_score}")

Original Score: -0.6114, Scaled Score: 19.429999999999996


In [20]:
sum_sentiment_score = 0

for comment in video_comments:
    
    sentiment = sia.polarity_scores(comment)
    
    # Scale compound score from [-1, 1] to [0, 100]
    scaled_score = (sentiment['compound'] + 1) * 50
    
    print(f'comment: {comment}, score: {scaled_score}')

    sum_sentiment_score += scaled_score

comment: Love you!, score: 83.48
comment: 16:32 just AWESOME!!!, score: 88.615
comment: It was me, silly 😛🤪☎️☎️, score: 51.29
comment: I can't tell you how happy I am,that we are connected again! Everything was restricted 😰👁️👁️🦋, score: 66.91
comment: Dear James,I got a new code 😊I am happy to find you here playing your wonderful guitar 🎸 i feel happy again 💜❤️, score: 96.91000000000001
comment: Wonderful to be back I can sleep with a smile 🥰😀🎸🦋, score: 86.75500000000001
comment: Just when I am almost giving up , you rescue me again 💞🎸💟🦋😀, score: 83.02499999999999
comment: I will always try to find James bay, my favourite human being 🎸💓😀, score: 50.0
comment: I love you beautiful ❤❤🇧🇷🇧🇷, score: 92.21
comment: I thought he was super high, then I realised I had playback speed set to 75%., score: 79.97000000000001
comment: You deserve my best 😘 smile on a busy Saturday 🙂🤣, score: 88.58500000000001
comment: I am always happy to see this lesson 😀😌, score: 78.595
comment: The first famous pe

In [19]:
sum_sentiment_score/len(video_comments)

69.12968354430377

In [21]:
len(video_comments)

158

## 3. Compare NLTJ to OpenAI (Just Testing, did not use OpenAI for Project)

In [27]:
#!pip install openai

In [100]:
API_KEY = 'API_KEY'

In [32]:
import openai

# Set your API key here
openai.api_key = API_KEY  # Replace with your actual OpenAI API key

# Using the chat model with the chat completions endpoint
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",  # Specify the chat model you're using
    messages=[
        {"role": "user", "content": "Analyze the sentiment of this comment: 'I love this video so much! 😍'"}
    ]
)

print(response.choices[0].message['content'])

The sentiment of the comment is positive.


In [44]:
# Playground

import openai

# Set your API key
openai.api_key = API_KEY

def get_sentiment_score(comment):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a sentiment analysis assistant. Provide a sentiment score from 0 (most negative) to 100 (most positive) for the given comment."},
            {"role": "user", "content": f"Comment: '{comment}'"}
        ]
    )
    # Extracting and processing the response to get the score
    response_content = response.choices[0].message['content']
    try:
        # Assuming the model returns a response like "Sentiment score: XX"
        score_str = response_content.split(":")[-1].strip()
        score = float(score_str)
        return score
    except ValueError:
        # In case the model response is not in the expected format
        return "Unable to determine sentiment score."

# Example usage
comment = "Please do stealing cars!!"
score = get_sentiment_score(comment)
print(f"Sentiment Score: {score}")


Sentiment Score: 10.0


In [ ]:
comment: Please do stealing cars!!, score: 27.205000000000002

^^this should be higher because comment indicates they really like the cover, and want the artist to do another song.

comment: I want nothing but to be able to sing like him, score: 76.335

comment: The first famous person I’ve ever seen that could actually teach, score: 50.0

comment: I 💝 U, score: 50.0

In [48]:
import openai

# Set your API key
openai.api_key = API_KEY

def get_sentiment_score(comment):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a sentiment analysis assistant specialized in analyzing YouTube comments on musical covers. Provide a sentiment score from 0 (most negative) to 100 (most positive) for the given comment."},
            {"role": "user", "content": f"Comment: '{comment}'"}
        ]
    )
    # Extracting and processing the response to get the score
    response_content = response.choices[0].message['content']
    print(response)
    try:
        # Assuming the model returns a response like "Sentiment score: XX"
        score_str = response_content.split(":")[-1].strip()
        score = float(score_str)
        return score
    except ValueError:
        # In case the model response is not in the expected format
        return "Unable to determine sentiment score."

# Example usage
comment = "Please do stealing cars!!"
score = get_sentiment_score(comment)
print(f"Sentiment Score: {score}")


{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Sentiment score: 60",
        "role": "assistant"
      }
    }
  ],
  "created": 1701648892,
  "id": "chatcmpl-8Rqto3pl5yWdJ1cGBVHt3KkVG5KQc",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 6,
    "prompt_tokens": 58,
    "total_tokens": 64
  }
}
Sentiment Score: 60.0


# xx

In [49]:
import pandas as pd
import uuid
import openai

# Function to get sentiment score using GPT-3.5
def get_sentiment_score(comment):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a sentiment analysis assistant."},
            {"role": "user", "content": f"Analyze the sentiment of this comment: '{comment}'"}
        ]
    )
    response_content = response.choices[0].message['content']
    try:
        score_str = response_content.split(":")[-1].strip()
        score = float(score_str)
        total_tokens = response.usage.total_tokens
        return score, total_tokens
    except ValueError:
        return None, None

# Function to process comments and store in DataFrame
def process_comments(video_id, comments):
    data = []
    for comment in comments:
        sentiment_score, total_tokens = get_sentiment_score(comment)
        if sentiment_score is not None:
            data.append({
                "Comment ID": str(uuid.uuid4()),
                "comment": comment,
                "video ID": video_id,
                "sentiment score": sentiment_score,
                "total_tokens": total_tokens
            })
    return pd.DataFrame(data)

# Example usage
video_id = '1U0-nCihPDI'
#video_comments = ["Love this video!", "Not a fan of this cover."]  # Replace with your list of comments
df = process_comments(video_id, video_comments)

print(df)

Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)

In [50]:
df

NameError: name 'df' is not defined

# 4. using NLTK

In [69]:
import pandas as pd
import uuid
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize VADER Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Function to get scaled sentiment score using VADER
def get_scaled_sentiment_score(comment):
    sentiment = sia.polarity_scores(comment)
    scaled_score = (sentiment['compound'] + 1) * 50  # Scaling to 0-100 range
    return scaled_score

# Function to process comments and store in DataFrame
def process_comments(video_id, comments):
    data = []
    for comment in comments:
        sentiment_score = get_scaled_sentiment_score(comment)
        data.append({
            "Comment ID": str(uuid.uuid4()),
            "comment": comment,
            "video ID": video_id,
            "sentiment score": sentiment_score,
        })
    return pd.DataFrame(data)

# 5. Generate sentiment files

In [96]:
import pandas as pd
import os

# Load the covers DataFrame
file_name = "youtube_covers.csv"
covers_df = pd.read_csv(file_name)

# first first 10 for now
covers_df = covers_df.head(n=10)

# Folder path where the CSV files will be saved
folder_path = 'data/sentiment/'

# Check if the folder exists, if not, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [97]:
covers_df

,video_id,upload_date,views,likes,dislikes,comments,subscribers,song_id
0,Duaxp1nc5po,2016-11-03T18:30:00Z,6342336.0,113146.0,NaN,1551.0,2310000,If I Ain't Got You - Alicia Keys
1,TnjS5kqyu3E,2022-11-02T01:05:46Z,1129167.0,22417.0,NaN,313.0,3870,If I Ain't Got You - Alicia Keys
2,jw3MqySX9qw,2021-10-01T10:00:23Z,1603208.0,48048.0,NaN,974.0,244000,If I Ain't Got You - Alicia Keys
3,vFny_UvyT0s,2023-05-24T14:00:40Z,409868.0,15201.0,NaN,516.0,681000,If I Ain't Got You - Alicia Keys
4,tq9nmRqNJZM,2015-02-05T11:01:15Z,6921650.0,77987.0,NaN,1218.0,25100,If I Ain't Got You - Alicia Keys
5,TOgOhMWA6oM,2023-02-02T18:38:39Z,181418.0,3090.0,NaN,90.0,39100,If I Ain't Got You - Alicia Keys
6,YJHs_P4voBM,2023-09-27T13:00:33Z,90855.0,5427.0,NaN,251.0,99600,If I Ain't Got You - Alicia Keys
7,jNV60kqZGNY,2019-03-27T14:11:30Z,197171.0,2811.0,NaN,102.0,53500,If I Ain't Got You - Alicia Keys
8,ddSoLkxzxyQ,2018-07-16T14:00:06Z,1304509.0,21579.0,NaN,527.0,1570000,If I Ain't Got You - Alicia Keys
9,g_xcxu0TTvo,2017-10-02T17:27:18Z,8357621.0,150908.0,NaN,3417.0,1140000,If I Ain't Got You - Alicia Keys


In [101]:
# # Example usage
# video_id = '1U0-nCihPDI'  # Replace with your video ID
# #video_comments = ["Love this video!", "Not a fan of this cover."]
# df = process_comments(video_id, video_comments)

# df

In [94]:
# # Process each cover and save to individual CSV files in the specified folder
# for index, row in covers_df.iterrows():
#     video_id = row['video_id']
#     video_comments = get_video_comments(video_id, max_results=1000)
#     sentiment_df = process_comments(video_id, video_comments)
#     file_path = os.path.join(folder_path, f'sentiment_{video_id}.csv')
#     sentiment_df.to_csv(file_path, index=False)

# Process each cover and save to individual CSV files in the specified folder
for index, row in covers_df.iterrows():
    video_id = row['video_id']
    song_id = row['song_id']  # Extract the song_id from covers_df

    video_comments = get_video_comments(video_id, max_results=1000)
    sentiment_df = process_comments(video_id, video_comments)

    # Add the song_id as a column to sentiment_df
    sentiment_df['song_id'] = song_id

    file_path = os.path.join(folder_path, f'sentiment_{video_id}.csv')
    sentiment_df.to_csv(file_path, index=False)


In [66]:
sum(df['sentiment score'])/len(df['sentiment score'])

69.12968354430377

# 6. Save data 

In [67]:
# Save DataFrame to CSV
df.to_csv('youtube_comments_sentiment.csv', index=False)